In [ ]:
import pandas as pd

# ------------------------------------------------------------
# 0. (Optional) Load DF_EV from CSV
#    - Assume DF_MAIN is already in memory.
#    - If DF_EV is already loaded, you can skip this block.
# ------------------------------------------------------------
# Example:
DF_EV = pd.read_csv("Data/EV_Pop_Growth_23_24.csv")

# ------------------------------------------------------------
# 1. Define placeholder column names (easy to change in one place)
#    - Replace these values with your real column names.
# ------------------------------------------------------------
MAIN_ZIP_COL = "Zip Code"  # ZIP column name in DF_MAIN
EV_ZIP_COL = "Zip Code"      # ZIP column name in DF_EV

# ------------------------------------------------------------
# 2. Normalize ZIP code columns in both dataframes
#    - Cast to string to preserve leading zeros (e.g., "00501")
#    - Strip whitespace just in case.
# ------------------------------------------------------------

DF_MAIN[MAIN_ZIP_COL] = (
    DF_MAIN[MAIN_ZIP_COL]
    .astype(str)
    .str.strip()
)

DF_EV[EV_ZIP_COL] = (
    DF_EV[EV_ZIP_COL]
    .astype(str)
    .str.strip()
)

# ------------------------------------------------------------
# 4. Filter DF_EV to only ZIPs that actually exist in DF_MAIN
#    - We expect not all ZIPs in DF_EV to appear in DF_MAIN.
#    - We drop any "orphan" ZIPs from DF_EV; they can't affect DF_MERGED.
# ------------------------------------------------------------
zips_in_main = DF_MAIN[MAIN_ZIP_COL].unique()
DF_EV_filtered = DF_EV_unique[DF_EV_unique[EV_ZIP_COL].isin(zips_in_main)].copy()

print(f"Unique ZIPs in DF_MAIN:       {len(zips_in_main):,}")
print(f"Rows in DF_EV before filter:  {len(DF_EV_unique):,}")
print(f"Rows in DF_EV after filter:   {len(DF_EV_filtered):,}")

# ------------------------------------------------------------
# 5. Perform the left merge on ZIP code
#    - Left: DF_MAIN (feeder-month-hour level)
#    - Right: DF_EV_unique (ZIP-level EV data)
#    - How this "blows out" EV data:
#        For each row in DF_MAIN, we look up that row's ZIP in DF_EV_unique
#        and attach all EV columns to the row, repeating the same EV values
#        for every feeder-month-hour with that ZIP.
# ------------------------------------------------------------
DF_MERGED = DF_MAIN.merge(
    DF_EV_unique,
    how="left",
    left_on=MAIN_ZIP_COL,
    right_on=EV_ZIP_COL,
    suffixes=("", "_EV"),  # DF_MAIN columns stay as-is, EV side gets "_EV" if conflicts
)

# Peek at the merged result
print("\nHead of DF_MERGED:")
print(DF_MERGED.head())
